In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1: Tina Pai
- Name 2: Ellen Lull
- Name 3: Paritosh Rai
- Name 4: Fabio Savorgnan

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from __future__ import print_function
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np


# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

*Double click to edit*


This is a dataset of news article topics. There are about 11,000 instances, each representing a news article. There are 20 different groups that are labeled in the dataset. The data will be very sparse, with only a small percentage of the data having a numeric value and all others being zero. 

This data was trained from twenty year old articles so many news topics would not generalize to topics in today's news. In fact, many words like 'facebook', 'twitter', and 'Obama' would not even be present in the existing dataset, but are important for modern topic classifiers to understand. However, some health related topics may still work well--it depends on how much the underlying distribution of words is changing.

This data is represented as a sparse matrix, meaning only non-zero values are stored with their matrix location.

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [3]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('Euclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:', euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:', jaccard(b>0,c>0))

print('\n\nThe most appropriate distance is...')
print('Cosine. It clearly delineates between topics effectively. B and C are closest (by far),',)
print('and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles
Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398236 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156


The most appropriate distance is...
Cosine. It clearly delineates between topics effectively. B and C are closest (by far),
and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

cv = StratifiedShuffleSplit(test_size = 0.5, train_size=0.5, random_state = 1).split(ds.data,ds.target)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
for K in [1, 3, 5, 7, 9, 11, 13]:
    clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')

    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)

    acc = accuracy_score(pred, y_test)
    
    
    print('Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))
    


#=====================================



Accuracy of classifier with 1 neighbors is: 0.60
Accuracy of classifier with 3 neighbors is: 0.53
Accuracy of classifier with 5 neighbors is: 0.50
Accuracy of classifier with 7 neighbors is: 0.48
Accuracy of classifier with 9 neighbors is: 0.46
Accuracy of classifier with 11 neighbors is: 0.45
Accuracy of classifier with 13 neighbors is: 0.43


The best accuracy is .6 with 1 neighbor. Even though having just 1 neighbor might risk overfitting, the increase in accuracy here is very substantial (7% higher than K=3) so we believe K=1 is the best value for K. It also makes sense that K=1 is good because the data is so sparse, that the next nearest neighbors may be much further away than the closest neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:
No, KD Trees don't work with sparse data because it would be hard to split the branches well when most of the values are 0; the tree would get very deep. Also, the cost of measuring the distance for sparse data is small, and the cost of creating the tree would be more than the cost of just measuring the distance between points. Therefore, the benefit of the KDTree is substantially reduced when data is sparse.

___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [11]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)



# fill in your code here
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# calculate the accuracy and print it for various values of K
for m in ['l1', 'l2', 'cosine']:
    clf = NearestCentroid(metric=m)

    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)

    acc = accuracy_score(pred, y_test)
    print("metric: ", m)
    print("acc:", acc)


print('The best distance metric is: Cosine')

/Users/tina/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


metric:  l1
acc: 0.3160685875905957


/Users/tina/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


metric:  l2
acc: 0.4157680749513877


/Users/tina/opt/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_nearest_centroid.py:150: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


metric:  cosine
acc: 0.49354781686406224
The best distance metric is: Cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

In [18]:
# Use this space for any calculations you might want to do


attributes = ds.data.shape[1]
classes = 20

print("Number of conditional probabilities:", attributes*classes)
print("Number of priors: 20")


Number of conditional probabilities: 2602140
Number of priors: 20


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

If the instances contain mostly continuous attributes, Gaussian Naive Bayes would be the best because Gaussian Naive Bayes is designed to support continuous features, as it finds the probability using the gaussian distribution to approximate the distribution of the features.

Yes, if the data is sparse, it does change our answer. If the data is sparse, we should use multinomial naive bayes because it would be faster. We can also use Bernoulli if the features are binary. It could be useful to binarize the features to reduce the complexity of the problem. Also it would not be appropriate to use Gaussian for sparse data because there would be so many 0s that a Gaussian distribution would not be a sensible approximation.

___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=1.0)
clf_bnb = BernoulliNB(alpha=1.0, binarize= 0.0)

# fill in your code here

for clf in [clf_mnb, clf_bnb]:
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc = accuracy_score(pred, y_test)
  
    print("clf:", clf)
    print("acc:", acc)


for a in [.0001, .001, .01, .05]:
    
    for b in [.001, .01, .02, .05]:
        clf_mnb = MultinomialNB(alpha=a)
        clf_bnb = BernoulliNB(alpha=a, binarize=b)

        for clf in [clf_mnb, clf_bnb]:
            clf.fit(X_train, y_train)
            pred = clf.predict(X_test)
            acc = accuracy_score(pred, y_test)

            print("clf:", clf)
            print("acc:", acc)

            
print('The best model parameters were: multinomial with alpha=.001, with an accuracy of .895')
print('These classifiers are so fast because... Naive Bayes learns with deterministic calculations and does not need do iterations and does not need to optimize a cost function and does not need to back-propagate errors, which makes it fast.')
print('The alpha values control... alpha is Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing), which affects how the model treats the problem of probabilities of classes being 0.')



clf: MultinomialNB()
acc: 0.7051440692946792
clf: BernoulliNB()
acc: 0.6162276825172353
clf: MultinomialNB(alpha=0.0001)
acc: 0.888103234930175
clf: BernoulliNB(alpha=0.0001, binarize=0.001)
acc: 0.8502739968181015
clf: MultinomialNB(alpha=0.0001)
acc: 0.888103234930175
clf: BernoulliNB(alpha=0.0001, binarize=0.01)
acc: 0.8714866537033763
clf: MultinomialNB(alpha=0.0001)
acc: 0.888103234930175
clf: BernoulliNB(alpha=0.0001, binarize=0.02)
acc: 0.8743150079547464
clf: MultinomialNB(alpha=0.0001)
acc: 0.888103234930175
clf: BernoulliNB(alpha=0.0001, binarize=0.05)
acc: 0.8348948205762772
clf: MultinomialNB(alpha=0.001)
acc: 0.8951741205586
clf: BernoulliNB(alpha=0.001, binarize=0.001)
acc: 0.8419657062047021
clf: MultinomialNB(alpha=0.001)
acc: 0.8951741205586
clf: BernoulliNB(alpha=0.001, binarize=0.01)
acc: 0.8683047551705851
clf: MultinomialNB(alpha=0.001)
acc: 0.8951741205586
clf: BernoulliNB(alpha=0.001, binarize=0.02)
acc: 0.8750220965175888
clf: MultinomialNB(alpha=0.001)
acc: 0.8

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.